In [3]:
import pandas as pd

# from pymystem3 import Mystem
# import nltk
# from nltk.tokenize import RegexpTokenizer
# import re
# # nltk.download('stopwords')
# from nltk.corpus import stopwords
# sw = stopwords.words('russian')

# tokenizer = RegexpTokenizer(r'\w+')
# m = Mystem()

In [4]:
books = pd.read_csv('lib_items.csv', sep = ';')  # sys_numb2title
books_v = books.values.tolist()
sys_numb2title = {}

for i in books_v:
    sys_numb2title[i[0]] = i[1]

In [5]:
trans = pd.read_csv('lib_train_transactions_extended.csv', sep = ';')  # chb2sys_nums
trans = trans.sort_values(by = 'chb')
trans_v = trans.values.tolist()
chb2sys_nums = {}

for i in trans_v:
    if i[0] in chb2sys_nums:
        chb2sys_nums[i[0]].append(i[1])
    else:
        chb2sys_nums[i[0]] = [i[1]]

In [ ]:
# def clear_and_morph(text, sw, m):  ## это код препроцессинга из первой попытки, все последующие разы (включая этот) -
#   text = str(text)                 ## я загружала файл
#   text = text.lower()
#   toks = tokenizer.tokenize(text)
#   f_text = ' '.join([i for i in toks if i not in sw])
#   lemmas = m.lemmatize(f_text)
#   return lemmas

In [ ]:
# titles_str = 'splittertoken'.join(books['title'].tolist())
# lemmas_titles = clear_and_morph(titles_str, sw, m)
# lemmatized_titles = ''.join(lemmas_titles).split('splittertoken')


In [6]:
lems = pd.read_csv('lib_lem_titles.csv')

In [7]:
lems = lems.drop(columns = ['Unnamed: 0'])

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
vectorizer = TfidfVectorizer()
X_tf = vectorizer.fit_transform(lems['0'].tolist())

In [10]:
from sklearn.neighbors import NearestNeighbors     

In [11]:
nbrs = NearestNeighbors(n_neighbors=20, algorithm='auto', metric='cosine')
nbrs.fit(X_tf)

NearestNeighbors(metric='cosine', n_neighbors=20)

In [21]:
users = pd.read_csv('lib_users.csv', sep = ';')

In [13]:
sys_num2idx = {}

In [14]:
cr = 0
for i in books.sys_numb:
    sys_num2idx[i] = cr
    cr += 1

In [15]:
sys_num2idx['RSL01005482081']

354354

In [1]:
# %%time
all_recs = {}

cr = 0
for i in users.chb:
    users_books = chb2sys_nums[i]
    
    if len(users_books) > 1:
        
        idx = sys_num2idx[users_books[-1]]
        n10 = nbrs.kneighbors(X_tf.getrow(idx), return_distance=False).tolist()[0][1:11]
        
        idx = sys_num2idx[users_books[-2]]
        n20 = nbrs.kneighbors(X_tf.getrow(idx), return_distance=False).tolist()[0][1:11]

        sub_recs = n10 + n20
        all_recs[i] = sub_recs
        
    else:
        idx = sys_num2idx[users_books[-1]]
        nbrs_list = nbrs.kneighbors(X_tf.getrow(idx.index[0]), return_distance=False).tolist()
        sub_recs = nbrs_list[0][1:21]   
        all_recs[i] = sub_recs
        


In [18]:
end_table = []
for i in all_recs:
    for b in all_recs[i]:
        el = [i, b]
        end_table.append(el)
        
end_df = pd.DataFrame(end_table, columns = ['chb', 'sys_numb'])

In [19]:
def decoder(text):
    sn = books.sys_numb[text]
    return sn

end_df['sys_numb'] = end_df['sys_numb'].apply(decoder)

In [20]:
end_df

,chb,sys_numb
0,300001020830,RSL01010055979
1,300001020830,RSL01005110672
2,300001020830,RSL01008627057
3,300001020830,RSL01006524186
4,300001020830,RSL01005575066
...,...,...
335055,300001155049,RSL01006037680
335056,300001155049,RSL01006208926
335057,300001155049,RSL01006219679
335058,300001155049,RSL01006268298


In [23]:
end_df.to_csv('lib_ent02.csv', sep = ';', index = False)